<a href="https://colab.research.google.com/github/devthumos/Processamento_de_Linguagem_Natural/blob/main/Atividade_avaliativa_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade Avaliativa 07

# Alunos: 
##### Gabriel Cesário Silva Martins -- 180100912
##### Luís Fernando Ferreira Pereira Lopes -- 200023187
##### Vitor Vasconcelos de Oliveira -- 180114778

### Teremos que utilizar o coeficiente Alfa de Krippendorf para encontrarmos a matriz de coincidência

In [ ]:
!pip install krippendorff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import re
import pandas as pd
import numpy as np
import os
import json
import krippendorff
from sklearn.metrics import cohen_kappa_score

# Leitura de arquivos exportados do tagtog 
# Retornando classificações dos 3 anotadores

In [ ]:
## Essa função retorna um dicionário cuja as chaves são os nomes dos usuários
# Os valores são dicionários em que as chaves são os arquivos json
def data_user(users: str, directory_members: str, debug=False) -> dict:
  dict_users = {user: {} for user in users}
  if debug:
    print("\t\tdict_data\n", dict_users)

  for user in users:
    user_directory = directory_members + r"/" + user + r"/pool"

    if debug:
      print(f"Collecting from {user_directory}")

    for filename in os.listdir(user_directory):
      file = os.path.join(user_directory, filename)

      if os.path.isfile(file):
        if debug:
          print("\t\t", get_json_filename(file))
        dict_users[user][get_json_filename(file)] = (retrieve_data(file))
    
  return dict_users


## Essa função retorna um dicionário em que as chaves são os usuários
# Os valores são listas das classificações de cada usuário
def data_annotators(dict_users: dict, main_user, debug=False) -> dict:
  dict_annotators = {user: [] for user in dict_users}

  for json_file in dict_users[main_user]:
    if debug:
      print(json_file)
    not_add = 0
    for user in dict_users:
      if user != main_user:
        if json_file not in dict_users[user]:
          not_add = 1
    if not_add == 0:
      for user in dict_users:
        dict_annotators[user].append(dict_users[user][json_file]["metas"]["m_8"]["value"])
    else:
      if debug:
        print(f"\to arquivo {json_file} não foi anotado por um dos dois anotadores")

  if debug:
    print(dict_annotators)

  return dict_annotators

## Essa função retorna apenas o nome do arquivo json sem o restante do seu caminho absoluto
def get_json_filename(file: str, debug=False):
  json_file = re.split(r"/", file)
  if debug:
    print(json_file[-1])

  return json_file[-1]

## Essa função retorna o conteúdo nos arquivos json como um dicionário
def retrieve_data(file, debug=False) -> dict:
  with open(file, "r", encoding="utf-8") as f:
    json_file = json.load(f)
  return json_file

In [ ]:
users = ["devthumos",
"VitinXD",
"g-cesario",]

In [ ]:
dict_users = data_user(users, "/content/drive/MyDrive/DataPLN/tagtog_ProjetoGabVi (2)/ProjetoGabVi/ann.json/members")

In [ ]:
dict_annotators = data_annotators(dict_users, "devthumos")

In [ ]:
for i in users:
  print(len(dict_annotators[i]))

50
50
50


# Criação do dataframe de classificações

In [ ]:
df = pd.DataFrame.from_dict(dict_annotators)

In [ ]:
df.head()

,devthumos,VitinXD,g-cesario
0,Positiva,Positiva,Positiva
1,Positiva,Positiva,Positiva
2,Neutra,Negativa,Neutra
3,Negativa,Neutra,Negativa
4,Negativa,Negativa,Negativa


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   devthumos  50 non-null     object
 1   VitinXD    50 non-null     object
 2   g-cesario  50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


# Tranformando Dataframe em Matriz de valores

In [ ]:
corpus_transpose = []

for atrib in df:
    lista = np.array(df[atrib])
    corpus_transpose.append(list(lista.T))

corpus_tranpose de forma resumida

In [ ]:
for i in range(len(corpus_transpose)):
  print(corpus_transpose[i][:5])

['Positiva', 'Positiva', 'Neutra', 'Negativa', 'Negativa']
['Positiva', 'Positiva', 'Negativa', 'Neutra', 'Negativa']
['Positiva', 'Positiva', 'Neutra', 'Negativa', 'Negativa']


In [ ]:
mapping = {'Positiva' : 2, 'Neutra' : 1, 'Negativa' : 0}

In [ ]:
corpus_transpose_int = []

for labels in corpus_transpose:
    t = []
    for doc in labels:
        t.append(mapping[doc])
    corpus_transpose_int.append(t)

corpus_tranpose_int de forma resumida

In [ ]:
for i in range(len(corpus_transpose_int)):
  print(corpus_transpose_int[i][:5])

[2, 2, 1, 0, 0]
[2, 2, 0, 1, 0]
[2, 2, 1, 0, 0]


# Coeficiente de Cohen's Kappa 

In [ ]:
print(cohen_kappa_score(corpus_transpose[0], corpus_transpose[1]))

0.7494631352899069


# Coeficiente de Krippendorff

In [ ]:
print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(reliability_data=corpus_transpose_int,
                                                                          level_of_measurement="nominal"))

print("Krippendorff's alpha for nominal metric: ", krippendorff.alpha(value_counts=corpus_transpose_int,
                                                                          level_of_measurement="nominal"))

Krippendorff's alpha for nominal metric:  0.8309014267185473
Krippendorff's alpha for nominal metric:  -0.009083469591353754
